In [31]:
import talib
import numpy as np
import pandas as pd
import math

In [37]:
def strided_app(origin_array: np.ndarray,
                window: int = 252, step=1) -> np.ndarray:
    if origin_array.size > window:
        nrows = ((origin_array.size - window) // step) + 1
    else:
        return origin_array
    n = origin_array.strides[0]
    return np.lib.stride_tricks.as_strided(origin_array,
                                           shape=(nrows, window),
                                           strides=(step * n, n))

def rolling_poly9(origin_array: np.ndarray, window: int = 252) -> np.ndarray:
    '''
    一次九项式滚动分解拟合
    '''
    index = range(window)

    def last_poly9(array_input):
        fit_params = np.polynomial.Chebyshev.fit(index, array_input, 9)
        return fit_params(index)[-1]

    if (len(origin_array) > window):
        stride_matrix = strided_app(origin_array, window, 1)
        # numpy.r_[]按照行方向拼接array，list是列向量形式存储，故仅能拼接array
        # numpy.c_[]按照列方向拼接array
        # .full()填充ndarray
        return np.r_[np.full(window - 1, np.nan),
                     np.array(list(map(last_poly9, stride_matrix)))]
    else:
        index = range(len(origin_array))
        fit_params = np.polynomial.Chebyshev.fit(index, origin_array, 9)
        y_fit_n = fit_params(index)
        return y_fit_n

In [ ]:
a = [4,3,2,1]
def inverse_num(series):
    # 因为numba不支持enumerate，所以后动计算逆序
    count = 0
    for i in range(len(series)-1):
        for j in range(i+1, len(series)):
            if series[i] > series[j]:
                count += 1
        i += 1
    return count

inverse_num(a)

In [83]:
df = pd.read_csv('data.csv', parse_dates=True, index_col=0)
def hma(array2hma: np.array, n: int = 10):
    return talib.WMA(2*talib.WMA(array2hma, int(n/2)) - talib.WMA(array2hma, n), int(math.sqrt(n)))

df['HMA10'] = hma(df['Close'].values)
# df['hma10'][~np.isnan(df['hma10'])]
# print(df['hma10'])
# print(df[hma10 != np.nan])

df['POLYNOMIAL9'] = np.r_[np.full(len(df['HMA10'][np.isnan(df['HMA10'])]),np.nan),rolling_poly9(df['HMA10'][~np.isnan(df['HMA10'])].values,252)]
print(df)

                 Open       High        Low  ...    Volume      HMA10  POLYNOMIAL9
 Date                                        ...                                  
2003-01-02   8.295000   8.830000   8.250000  ...  19640400        NaN          NaN
2003-01-03   8.750000   9.175000   8.675000  ...  15090600        NaN          NaN
2003-01-06   8.925000   9.595000   8.890000  ...  21209400        NaN          NaN
2003-01-07   8.935000   9.650000   8.930000  ...  28092600        NaN          NaN
2003-01-08   9.435000   9.650000   9.280000  ...  19244600        NaN          NaN
...               ...        ...        ...  ...       ...        ...          ...
2005-12-23  41.090000  41.099998  40.450001  ...   5070200  40.554567    39.862968
2005-12-27  40.650002  40.939999  39.849998  ...  11672900  40.270890    39.662767
2005-12-28  40.099998  40.480000  39.770000  ...  11567900  40.106364    39.479761
2005-12-29  40.250000  40.349998  39.410000  ...  10116600  39.875950    39.295612
2005